## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Generating A/B Menu items using Gemini Pro and ImageGen2.


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will create new menu desciption and images that allows different trucks to test different menu descriptions and images.  The notebook also shows how we can create the new menu table by using Gemini Pro to read an ERD.

- Notebook Logic:
    1. Create our new A/B menu table by using Gemini Pro to read our ERD and create our table along with primary keys. 
        1. First we ened to download a picture of our ERD.
        2. We need to construct a LLM prompt telling it to read the ERD and construct our SQL.
        3. Execute the SQL to create the new table and primary keys on the table.
    2. Select some locations (trucks)
    3. Run a query that sums Oct and Nov sales data by menu item.
        - Rank the sales (high drop off) for each city / location / menu item.
        - We will generate a new menu purmutation for the item that has the highest drop in sales
    4. Create our LLM prompt:
        - We want to generate new data for each of the menu items based upon our BigQuery results
        - The prompt will:
            - Provide the stating primary key
            - Ask for synthetic data to be generated
            - Provide a Google Cloud Storage pattern of the image of the menu item
            - We will ask the LLM to write our ImageGen2 LLM prompt
            - We will pass in the schema of the table
    5. Execute the SQL adding rows to our BigQuery table
    6. Query the new menu items, retrieving the LLM image prompt
    7. Generate the menu item image using ImageGen2.
    8. Upload the image to GCS.  The path will match what the LLM generated for the path.

## Initialize Python

In [ ]:
project_id="${project_id}"
location="us-central1"
model_id = "imagegeneration@005"

# No need to set these
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
city_languages=["American English", "British English", "Japanese", "American English"]
number_of_coffee_trucks = "4"

dataset_id = "data_beans_synthetic_data"

gcs_storage_bucket = "${data_beans_curated_bucket}"
gcs_storage_path = "data-beans/menu-images-a-b-testing/"

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2

from google.cloud import bigquery
client = bigquery.Client()

## ImageGen2 / Gemini Pro / Gemini Pro Vision (Helper Functions)

#### ImageGen2

In [ ]:
def ImageGen(prompt):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/imagegeneration:predict"

  payload = {
    "instances": [
      {
        "prompt": prompt
      }
    ],
    "parameters": {
      "sampleCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
    image_data = base64.b64decode(image_data)
    filename= str(uuid.uuid4()) + ".png"
    with open(filename, "wb") as f:
      f.write(image_data)
    print(f"Image generated OK.")
    return filename
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Gemini Pro LLM

In [ ]:
def GeminiProLLM(prompt, temperature = .8, topP = .8, topK = 40):

  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro:streamGenerateContent"

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 8192,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      try:
        llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]
      except Exception as err:
        print(f"response.content: {response.content}")
        raise RuntimeError(err)

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


#### Gemini Pro Vision LLM

In [ ]:
# Use the Gemini with Vision
def GeminiProVisionLLM(prompt, imageBase64, temperature = .4, topP = 1, topK = 32):

  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro-vision:streamGenerateContent"

  payload = {
  "contents": [
      {
        "role": "user",
        "parts": [
          {
            "text": prompt
          },
          {
            "inlineData": {
              "mimeType": "image/png",
              "data": f"{imageBase64}"
            }
          }
        ]
      }
    ],
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 2048,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


In [ ]:
# Use the Gemini with Vision
def GeminiProVisionMultipleFileLLM(prompt, image_prompt, temperature = .4, topP = 1, topK = 32):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro-vision:streamGenerateContent"


  parts = []
  new_item = {
      "text": prompt
      }
  parts.append(new_item)

  for item in image_prompt:
    new_item = {
        "text": f"Image Name: {item['llm_image_filename']}:\n"
        }
    parts.append(new_item)
    new_item = {
        "inlineData": {
            "mimeType": "image/png",
            "data": item["llm_image_base64"]
            }
        }
    parts.append(new_item)

  payload = {
  "contents": [
      {
        "role": "user",
        "parts": parts
      }
    ],
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 2048,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


#### SQL Functions

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

In [ ]:
def GetTableSchema(dataset_name, table_name):
  import io

  dataset_ref = client.dataset(dataset_name, project=project_id)
  table_ref = dataset_ref.table(table_name)
  table = client.get_table(table_ref)

  f = io.StringIO("")
  client.schema_to_json(table.schema, f)
  return f.getvalue()

In [ ]:
def GetStartingValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) + 1 AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetForeignKeys(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(CAST({field_name} AS STRING), "," ORDER BY {field_name}) AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetDistinctValues(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(DISTINCT {field_name}, "," ) AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
# Run DDL
# This has been done to make the code re-runnable for the demo
# We need to add some logic for the PK and FKs and the prompt will be updated to code gen it

def RunDDL(sql):
  import time

  sql = f"""CREATE SCHEMA IF NOT EXISTS `{project_id}.{dataset_id}`;

CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.menu_a_b_testing`
(
    menu_a_b_testing_id INT64 NOT NULL OPTIONS(description="Primary key. Menu A/B Testing table."),
    menu_id INT64 NOT NULL OPTIONS(description="Foreign key: Menu table."),
    location_id INT64 NOT NULL OPTIONS(description="Foreign key: Location table."),
    item_name STRING NOT NULL OPTIONS(description="The name of the item."),
    item_description STRING NOT NULL OPTIONS(description="The description of the item."),
    item_size STRING NOT NULL OPTIONS(description="The size of the item."),
    item_price FLOAT64 NOT NULL OPTIONS(description="The price of the item."),
    llm_item_description_prompt STRING OPTIONS(description="The prompt used to generate the LLM item description."),
    llm_item_description STRING OPTIONS(description="The LLM generated description of the item."),
    llm_item_image_prompt STRING OPTIONS(description="The prompt used to generate the LLM item image."),
    llm_item_image_url STRING OPTIONS(description="The LLM generated image url of the item."),
    create_date TIMESTAMP NOT NULL OPTIONS(description="The date the item was created."),
    llm_marketing_prompt STRING OPTIONS(description="The prompt used to generate the LLM marketing response."),
    llm_marketing_response JSON OPTIONS(description="The LLM generated marketing response."),
    llm_marketing_parsed_response JSON OPTIONS(description="The parsed LLM generated marketing response."),
    html_generated BOOLEAN OPTIONS(description="True if the HTML was generated."),
    html_filename STRING OPTIONS(description="The name of the HTML file."),
    html_url STRING OPTIONS(description="The URL of the HTML file.")
)
CLUSTER BY menu_a_b_testing_id;


ALTER TABLE `{project_id}.{dataset_id}.menu_a_b_testing` DROP PRIMARY KEY IF EXISTS;
ALTER TABLE `{project_id}.{dataset_id}.menu_a_b_testing` ADD PRIMARY KEY (menu_a_b_testing_id) NOT ENFORCED;
  """

  # To see the contraints in BigQuery
  # SELECT * FROM  `{project_id}.{dataset_id}.INFORMATION_SCHEMA.TABLE_CONSTRAINTS`


  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(2)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

#### Helper Functions

In [ ]:
def convert_png_to_base64(image_path):
  image = cv2.imread(image_path)

  # Convert the image to a base64 string.
  _, buffer = cv2.imencode('.png', image)
  base64_string = base64.b64encode(buffer).decode('utf-8')

  return base64_string

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

## Menu Synthetic Data and Image Generation

#### Download the ERD image and Generate our Database Schema DDL

In [ ]:
import requests
import shutil

menu_erd_filename = "Data-Beans-Menu-A-B-Testing-ERD.png"

# Specify the image URL
img_url = f"https://storage.googleapis.com/data-analytics-golden-demo/data-beans/v1/colab-supporting-images/{menu_erd_filename}"

# Send a GET request to fetch the image
response = requests.get(img_url, stream=True)

# Check for successful download
if response.status_code == 200:
    # Set decode_content to True to prevent encoding errors
    response.raw.decode_content = True

    # Open a local file in binary write mode
    with open(menu_erd_filename, "wb") as f:
        # Copy image data to the local file in chunks
        shutil.copyfileobj(response.raw, f)

    print("Image downloaded successfully!")
else:
    print("Image download failed with status code:", response.status_code)

In [ ]:
print(f"Filename: {menu_erd_filename}")
img = Image.open(menu_erd_filename)
img.thumbnail([504,700]) # width, height
IPython.display.display(img)

In [ ]:
llm_erd_prompt=f"""Use BigQuery SQL commands to create the following:
- Create a new BigQuery schema named "{dataset_id}".
- Use only BigQuery datatypes.  Double and triple check this since it causes a lot of errors.
- Create the BigQuery DDLs for the attached ERD.
- Create primary keys  for each table using the ALTER command. Use the "NOT ENFORCED" keyword.
- DO NOT Create foreign keys for each table.
- For each field add an OPTIONS for the description.
- Cluster the table by the primary key.
- For columns that can be null do not add "NULL" to the create table statement.  BigQuery leaves this blank.  For example this is INCORRECT: STRING NULL OPTIONS.
- All ALTER TABLE statements should by at the bottom of the generated script.
- The ALTER TABLES statements should be order by the primary key statements and then the foreign key statements. Order matters!
- Double check your work especially that you used ONLY BigQuery data types.
- Double check that NULL was not specified for NULLABLE fields.
- Only create the tables shown in the diagram. Do not create foreign key tables.

Previous Errors that have been generated by this script.  Be sure to check your work to avoid encountering these.
- Query error: Type not found: FLOAT at [6:12]
- Query error: Table test.company does not have Primary Key constraints
- Query error: Syntax error: Expected ")" or "," but got keyword NULL

Example:
CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.customer`
(
    customer_id INTEGER NOT NULL OPTIONS(description="Primary key. Customer table."),
    country_id INTEGER NOT NULL OPTIONS(description="Foreign key: Country table."),
    customer_llm_summary STRING NOT NULL OPTIONS(description="LLM generated summary of customer data."),
    customer_lifetime_value STRING NOT NULL OPTIONS(description="Total sales for this customer."),
    customer_cluster_id FLOAT NOT NULL OPTIONS(description="Clustering algorithm id."),
    customer_review_llm_summary STRING  OPTIONS(description="LLM summary are all of the customer reviews."),
    customer_survey_llm_summary STRING  OPTIONS(description="LLM summary are all of the customer surveys.")
)
CLUSTER BY customer_id;

CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.country`
(
country_id INTEGER NOT NULL OPTIONS(description="Primary key. Country table."),
country_name STRING NOT NULL OPTIONS(description="The name of the country.")
)
CLUSTER BY country_id;


ALTER TABLE `{project_id}.{dataset_id}.customer` ADD PRIMARY KEY (customer_id) NOT ENFORCED;
ALTER TABLE `{project_id}.{dataset_id}.country` ADD PRIMARY KEY (country_id) NOT ENFORCED;
"""

In [ ]:
# Run the LLM to generate the DDL and run the DDL
imageBase64 = convert_png_to_base64(menu_erd_filename)

llm_success = False
temperature=.2
while llm_success == False:
  try:
    sql = GeminiProVisionLLM(llm_erd_prompt, imageBase64, temperature=temperature, topP=1, topK=32)
    # Need to prompt this
    sql = sql.replace("STRING NULL OPTIONS","STRING OPTIONS")
    sql = sql.replace("JSON NULL OPTIONS","JSON OPTIONS")
    sql = sql.replace("BOOLEAN NULL OPTIONS","BOOLEAN OPTIONS")
    print(f"SQL: {sql}")
    llm_success = RunDDL(sql)
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

### Find the location you want to update

In [ ]:
# Set your location ids

location_ids = [2,43,10,30]
# location_id = 1 # CitySips Roaming Cafe(New York City)
# location_id = 2 # JavaJourney Express(London)
# location_id = 3 # UrbanCaffeine Cruiser(London)
# location_id = 4 # CafeWheels(Tokyo)
# location_id = 5 # Golden Gate Grind Mobile(San Francisco)
# location_id = 6 # Sunshine Sips on Wheels(San Francisco)
# location_id = 7 # Big Apple Brew Bus(London)
# location_id = 8 # Bay Brew Hauler(San Francisco)
# location_id = 9 # Magic City Mocha Mobile(Tokyo)
# location_id = 10 # Metropolis Mug Mover(Tokyo)
# location_id = 11 # Nectar Nomad(Tokyo)
# location_id = 12 # Street Sips(New York City)
# location_id = 13 # MiaMornings Mobile(London)
# location_id = 14 # CityBeans Roam-uccino(New York City)
# location_id = 15 # Sunrise City Sipper(Tokyo)
# location_id = 16 # Gotham Grind on Wheels(New York City)
# location_id = 17 # Bay Area Bean Bus(San Francisco)
# location_id = 18 # Mia Mochaccino Mobile(London)
# location_id = 19 # Cityscape Sip Stop(Tokyo)
# location_id = 20 # Transit Brew Buggy(London)
# location_id = 21 # Fog City Fueler(London)
# location_id = 22 # Metro Mugs(London)
# location_id = 23 # Espresso Express(Tokyo)
# location_id = 24 # Sunny Side Sips Shuttle(London)
# location_id = 25 # Empire Espresso Explorer(New York City)
# location_id = 26 # SF Sidewalk Sipper(San Francisco)
# location_id = 27 # Beachside Brew Bounder(San Francisco)
# location_id = 28 # Urban Sipper's Shuttle(London)
# location_id = 29 # Nomadic Nectar(London)
# location_id = 30 # Golden Bridge Brewmobile(San Francisco)
# location_id = 31 # Sunny State Sipster(San Francisco)
# location_id = 32 # Cafe Cruiser Central(Tokyo)
# location_id = 33 # Neighborhood Nectar(Tokyo)
# location_id = 34 # Frisco Fuel on Wheels(Tokyo)
# location_id = 35 # MiaMug Mobility(New York City)
# location_id = 36 # Metropolitan Mochaccino(London)
# location_id = 37 # CitySips Street Surfer(New York City)
# location_id = 38 # Golden Gate Gourmet Glide(San Francisco)
# location_id = 39 # Beach Breeze Brew Bus(San Francisco)
# location_id = 40 # City Roast Cruiser(Tokyo)
# location_id = 41 # Urban Uplifter(New York City)
# location_id = 42 # Frisco Fresh Brews(New York City)
# location_id = 43 # Magic Mugs(London)
# location_id = 44 # Coffee Cart Connection(New York City)
# location_id = 45 # Empire City Espresso Explorer(New York City)
# location_id = 46 # Golden Glow Grind Rover(San Francisco)
# location_id = 47 # Sun-Kissed Sip & Go(San Francisco)
# location_id = 48 # CityLife Latte Lorry(New York City)
# location_id = 49 # Cityscape Sipper Shuttle(Tokyo)
# location_id = 50 # Golden Grind Getter(San Francisco)

In [ ]:
location_id_string = ' '.join(map(str, location_ids))

sql = f"""WITH oct_data AS
(
SELECT order_item.menu_id,
       order_t.location_id,
       CAST(SUM(order_item.item_total) AS INT64) AS sum_item_total
  FROM `${project_id}.data_beans_curated.order_item` AS order_item
        INNER JOIN `${project_id}.data_beans_curated.order` AS order_t
                ON order_t.order_id = order_item.order_id
               AND order_t.order_datetime BETWEEN '2023-10-01'
                                              AND '2023-10-31'
GROUP BY 1,2
)
, nov_data AS
(
SELECT order_item.menu_id,
       order_t.location_id,
       CAST(SUM(order_item.item_total) AS INT64) AS sum_item_total
  FROM `${project_id}.data_beans_curated.order_item` AS order_item
        INNER JOIN `${project_id}.data_beans_curated.order` AS order_t
                ON order_t.order_id = order_item.order_id
               AND order_t.order_datetime BETWEEN '2023-11-01'
                                              AND '2023-11-30'
GROUP BY 1,2
)
, results AS
(
SELECT city.city_name,
       nov_data.location_id,
       location.location_name,
       menu.*,
       nov_data.sum_item_total AS current_month_sales,
       oct_data.sum_item_total AS prior_month_sales,
       (oct_data.sum_item_total - nov_data.sum_item_total) AS sales_drop_off,
       ROW_NUMBER() OVER (PARTITION BY city.city_name, nov_data.location_id ORDER BY (oct_data.sum_item_total - nov_data.sum_item_total)) AS ranking
  FROM nov_data
       INNER JOIN `${project_id}.data_beans_curated.menu` AS menu
               ON menu.menu_id = nov_data.menu_id
       INNER JOIN `${project_id}.data_beans_curated.location` AS location
               ON location.location_id = nov_data.location_id
       INNER JOIN `${project_id}.data_beans_curated.city` AS city
               ON city.city_id = location.city_id
        LEFT JOIN oct_data
               ON nov_data.menu_id = oct_data.menu_id
              AND nov_data.location_id = oct_data.location_id
)
SELECT *
FROM results AS t
WHERE ranking = 1
  AND location_id IN ({str(location_ids).replace('[','').replace(']','') })  -- comment this out to see all locations
ORDER BY city_name, location_id, ranking;
"""

In [ ]:
df_low_sales_item = client.query(sql).to_dataframe()
items_to_generate = []

for row in df_low_sales_item.itertuples():
  items_to_generate.append ({
      "city_name" : row.city_name,
      "current_month_sales" : row.current_month_sales,
      "prior_month_sales" : row.prior_month_sales,
      "sales_drop_off" : row.sales_drop_off,
      "menu_id" : row.menu_id,
      "item_name" : row.item_name,
      "company_id" : row.company_id,
      "location_id" : row.location_id,
      "location_name" : row.location_name,
      "item_price" : row.item_price,
      "item_description" : row.item_description,
      "item_size" : row.item_size,
      "llm_item_description_prompt" : row.llm_item_description_prompt,
      "llm_item_description" : row.llm_item_description,
      "llm_item_image_prompt" : row.llm_item_image_prompt,
      "llm_item_image_url" : row.llm_item_image_url,
  })

for item in items_to_generate:
  print(f"location_id:                 {item['location_id']}")
  print(f"location_name:               {item['location_name']}")
  print(f"city_name:                   {item['city_name']}")
  print(f"current_month_sales:         {item['current_month_sales']}")
  print(f"prior_month_sales:           {item['prior_month_sales']}")
  print(f"sales_drop_off:              {item['sales_drop_off']}")
  print("")
  print(f"menu_id:                     {item['menu_id']}")
  print(f"item_name:                   {item['item_name']}")
  print(f"company_id:                  {item['company_id']}")
  print(f"location_id:                 {item['location_id']}")
  print(f"item_price:                  {item['item_price']}")
  print(f"item_description:            {item['item_description']}")
  print(f"item_size:                   {item['item_size']}")
  print(f"llm_item_description_prompt: {item['llm_item_description_prompt']}")
  print(f"llm_item_description:        {item['llm_item_description']}")
  print(f"llm_item_image_prompt:       {item['llm_item_image_prompt']}")
  print(f"llm_item_image_url:          {item['llm_item_image_url']}")


### Generate new images

In [ ]:
table_name = "menu_a_b_testing"
primary_key = "menu_a_b_testing_id"

schema = GetTableSchema(dataset_id, table_name)
starting_value = GetStartingValue(dataset_id, table_name, primary_key)
current_value = starting_value

for item in items_to_generate:
  menu_names_sql_prompt=f"""
  You are a database engineer and need to generate data for a table for the below schema.
  - The schema is for a Google Cloud BigQuery Table.
  - The table name is "{project_id}.{dataset_id}.{table_name}".
  - Read the description of each field for valid values.
  - Do not preface the response with any special characters or 'sql'.
  - Generate 1 rows of data for this table.
  - The starting value of the field {primary_key} is {current_value}.
  - Only generate a single statement, not multiple INSERTs.
  - Hardcode the field "menu_id" to the value of "{item['menu_id']}".
  - Hardcode the field "location_id" to the value of "{item['location_id']}".
  - Hardcode the field "item_name" to the value of "{item['item_name']}".
  - Hardcode the field "company_id" to the value of "{item['company_id']}".
  - Hardcode the field "item_price" to the value of "{item['item_price']}".
  - Hardcode the field "item_size" to the value of "{item['item_size']}".
  - Hardcode the field "menu_id" to the value of "{item['menu_id']}".
  - For the field "llm_item_image_prompt", limit the text to 256 characters.
  - For the field "llm_item_image_prompt": Think outside the box that encourages unconventional approaches and fresh perspectives based upon the item description.
  - For the field "llm_item_image_url" use the following pattern and replace [[menu_a_b_testing_id]] with the generated menu id: https://storage.cloud.google.com/{gcs_storage_bucket}/{gcs_storage_path}[[menu_a_b_testing_id]].png

  Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
  Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

  Schema: {schema}
  """

  llm_success = False
  temperature=.8
  while llm_success == False:
    try:
      sql = GeminiProLLM(menu_names_sql_prompt, temperature=temperature, topP=.8, topK = 40)
      print(f"SQL: {sql}")
      llm_success = RunQuery(sql)
      if llm_success == True:
        current_value = current_value + 1
    except:
      # Reduce the temperature for more accurate generation
      temperature = temperature - .05
      print("Regenerating...")


In [ ]:
# Query to get a list of menu items and the prompts for the images

sql = f"""SELECT menu_a_b_testing_id,
                 item_name,
                 llm_item_image_prompt
          FROM `{project_id}.{dataset_id}.{table_name}`
         WHERE menu_a_b_testing_id BETWEEN {starting_value} AND {current_value - 1}
        ORDER BY menu_id"""

print(f"SQL: {sql}")
df_process = client.query(sql).to_dataframe()
image_files = []

for row in df_process.itertuples():
  menu_a_b_testing_id = row.menu_a_b_testing_id
  item_name = row.item_name
  llm_item_image_prompt = row.llm_item_image_prompt

  print(f"item_name: {item_name}")
  print(f"llm_item_image_prompt: {llm_item_image_prompt}")
  try:
    image_file = ImageGen(llm_item_image_prompt)
    image_files.append ({
          "menu_a_b_testing_id" : menu_a_b_testing_id,
          "item_name" : item_name,
          "llm_item_image_prompt" : llm_item_image_prompt,
          "gcs_storage_bucket" : gcs_storage_bucket,
          "gcs_storage_path" : gcs_storage_path,
          "llm_image_filename" : image_file
      })
  except:
    print("Image failed to generate.")

In [ ]:
# View the results
for item in image_files:
  print(f"menu_a_b_testing_id: {item['menu_a_b_testing_id']}")
  print(f"item_name: {item['item_name']}")
  print(f"llm_item_image_prompt: {item['llm_item_image_prompt']}")
  img = Image.open(item["llm_image_filename"])
  img.thumbnail([500,500])
  IPython.display.display(img)

#### Save the results to storage

In [ ]:
# When we create the sample data for our table we also asked for the LLM to generate the correct GCS / HTTP path

# Copy all image files to storage
for item in image_files:
  copy_file_to_gcs(item["llm_image_filename"],item["gcs_storage_bucket"], item["gcs_storage_path"] + str(item['menu_a_b_testing_id']) + ".png")